# UNICEF Ghana: Handwashing with Ananse

Taking a look at time distributions of the video data

[Project Homepage](https://www.unicef.org/ghana/communication_10253.html)

[GitHub Repository](https://github.com/austinmw/ec520_ghana_handwashing)

[nbviewer](http://nbviewer.jupyter.org/github/austinmw/ec520_ghana_handwashing/blob/master/Timestamp%20Summary%20Statistics.ipynb)


** Tested for Mac, probably works with other Unix-based systems as well**

First I performed some very basic cleaning of the directories:
- Tried to standardize the spacing in file names
- Fixed outlier errors in naming conventions for some entries
- Added some empty directories where missing (Video 2) to make directory path error checking more straight forward

## 1: Import Libraries and Data

In [12]:
# imports
import numpy as np
print np.__version__
import pandas as pd
print pd.__version__
import matplotlib as mpl
print mpl.__version__
import matplotlib.pyplot as plt
#print plt.__version__
import re
print re.__version__
import seaborn as sns
print sns.__version__
import os
import os.path, time
import sys
%matplotlib inline
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

os.listdir('/Volumes/GhanaStudyData/BaselineVideosForAnalysis/')


1.11.3
0.19.2
2.0.0
2.2.1
0.7.1


['.DS_Store',
 'Agona Nkran Islamic',
 'Akakom DA Primary',
 'Akropong Presby Primary',
 'Akwaboso Methodist Primary',
 'Ampampatia',
 'Asafo Presby Primary',
 'Asamama Presby Primary',
 'Awutu AME Zion',
 'Ayinam Presby Primary',
 'Gyesikrom AEDA Primary',
 'Itifaqiya Islamic Primary',
 'Kwabeng Anglican Primary',
 'Mankrong Methodist Primary',
 'Ninta Methodist',
 'Nsabaa AME Zion',
 'Nsabaa Methodist',
 'Nuriya DA Primary Bawijiase',
 'Presby Primary Bawjiase',
 'Rev Erickson',
 'St Killian Catholic Primary']

In [13]:
# Toggle 1st/2nd hard drive

hdpath = '/Volumes/GhanaStudyData/BaselineVideosForAnalysis/' 
    
# list schools
schools = [f for f in os.listdir(hdpath) if f[0]!='.']
print(schools)
numSchools = len(schools)
print (numSchools)
#print("School Names:",*schools, sep='\n')

['Agona Nkran Islamic', 'Akakom DA Primary', 'Akropong Presby Primary', 'Akwaboso Methodist Primary', 'Ampampatia', 'Asafo Presby Primary', 'Asamama Presby Primary', 'Awutu AME Zion', 'Ayinam Presby Primary', 'Gyesikrom AEDA Primary', 'Itifaqiya Islamic Primary', 'Kwabeng Anglican Primary', 'Mankrong Methodist Primary', 'Ninta Methodist', 'Nsabaa AME Zion', 'Nsabaa Methodist', 'Nuriya DA Primary Bawijiase', 'Presby Primary Bawjiase', 'Rev Erickson', 'St Killian Catholic Primary']
20


## 2: Extracting the Timestamps

According to stackoverflow, retrieving file creation dates turns out to be platform-dependent.   
Testing this with Mac, although it may work with other Unix-based systems.  

In [14]:
# List of all video file paths in each school's Video/Video 1 and Video/Video 2 folders

vidPaths = [hdpath+s+'/All/' for s in schools]


print (vidPaths)
#vidPaths = [hdpath+s for s in schools] 

#if not hd2:
#  vidPaths = [hdpath+s+'/Baseline/Video/Video 1/' for s in schools] 
#  vidPaths += [hdpath+s+'/Baseline/Video/Video 2/' for s in schools]
#else:
 #   vidPaths = [hdpath+s+'/Video 1/' for s in schools]
#  vidPaths += [hdpath+s+'/Video 2/' for s in schools]
    

# convert school names to proper linux directory paths (might need later)
# escapedTopVidDirs = [hdpath+s.replace(' ', r'\ ')+'/Baseline/Video/' for s in schools]
# escaped list of paths for 'Video 1' and 'Video 2' (also may need later)
# escapedBothVidDirs = [e+'Video\\ 1/' for e in escapedTopVidDirs] + [e+'Video\\ 2/' for e in escapedTopVidDirs]

# Create large list of full file paths for each video in each school's Video 1 and Video 2 directories
vidFullPaths = [f+v for f in vidPaths for v in os.listdir(f) if v[0]!='.']



vidNum = len(vidFullPaths)
print("Number of total videos in all 'Video 1' and 'Video 2' directories: ", vidNum)

# list(each school) of lists(each vid path) for each 'Video 1' and 'Video 2' dir
v1FullPaths = [[s+v for v in os.listdir(s) if v[0]!='.'] for s in vidPaths[:int(len(vidPaths)/2)]]
v2FullPaths = [[s+v for v in os.listdir(s) if v[0]!='.'] for s in vidPaths[int(len(vidPaths)/2):]]
# aggregated above 
bothFullPaths = [v1FullPaths[i]+v2FullPaths[i] for i in range(len(v1FullPaths))] # 16730 total (hd1)

['/Volumes/GhanaStudyData/MasterTest/Agona Nkran Islamic/All/', '/Volumes/GhanaStudyData/MasterTest/Akakom DA Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Akropong Presby Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Akwaboso Methodist Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Ampampatia/All/', '/Volumes/GhanaStudyData/MasterTest/Asafo Presby Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Asamama Presby Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Awutu AME Zion/All/', '/Volumes/GhanaStudyData/MasterTest/Ayinam Presby Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Gyesikrom AEDA Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Itifaqiya Islamic Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Kwabeng Anglican Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Mankrong Methodist Primary/All/', '/Volumes/GhanaStudyData/MasterTest/Ninta Methodist/All/', '/Volumes/GhanaStudyData/MasterTest/Nsabaa AME Zion/All/', '/Volumes/GhanaStudyData/MasterTest/Nsab

In [15]:
# list of lists with creation times for all videos in all 'Video 1' and 'Video 2' directories
timestamps = [time.ctime(os.path.getmtime(f)).replace('  ', ' ').split(' ') for f in vidFullPaths]

# list of lists of lists: sTimestamps[school][timestamp][section of timestamp]
sTimestamps = [[time.ctime(os.path.getmtime(v)).replace('  ', ' ').split(' ') for v in school] for school in bothFullPaths] 

# example of first 10 videos and corresponding timestamps
print("Example of full file paths and creation dates for the first five videos:\n")
for i in range(3):
    print('path:\t    ' + vidFullPaths[i],'\ntimestamp: ',timestamps[i],'\n')

Example of full file paths and creation dates for the first five videos:

('path:\t    /Volumes/GhanaStudyData/MasterTest/Agona Nkran Islamic/All/ 07_Feb_22_2017--Agona_Nkran_Islamic279063563.avi', '\ntimestamp: ', ['Wed', 'Feb', '22', '07:45:20', '2017'], '\n')
('path:\t    /Volumes/GhanaStudyData/MasterTest/Agona Nkran Islamic/All/ 07_Mar_01_2017--Agona_Nkran_Islamic494214773.avi', '\ntimestamp: ', ['Wed', 'Mar', '1', '07:35:08', '2017'], '\n')
('path:\t    /Volumes/GhanaStudyData/MasterTest/Agona Nkran Islamic/All/ 07_Mar_01_2017--Agona_Nkran_Islamic631361197.avi', '\ntimestamp: ', ['Wed', 'Mar', '1', '07:26:24', '2017'], '\n')


# 2.5: backup data

Backing up all HD1 paths/mtimes/filenames to .csv and .pkl files 

Can do statistics on this dataframe in new notebook rather than requiring HD   
(all code in this notebook still requires the HD, I wrote it before I did this backup step)

In [16]:
# Put all file info in data frame
hdFiles = pd.DataFrame([f.split('/') for f in vidFullPaths])

# check hd since they have different paths
#if not hd2:
#hdFiles = hdFiles[[3,4,7,8]]
#else:
hdFiles = hdFiles[[3,4,5,6]]
    
hdFiles.columns = ['hard drive','school','video folder', 'file name']
hdFiles.head()
timestamps = pd.DataFrame([time.ctime(os.path.getmtime(f)).replace('  ', ' ').split(' ') for f in vidFullPaths], 
                          columns=['day','month','date','time','year'])
hdDataframe = hdFiles.join(timestamps)


# time to float, e.g. 10:06:30 -> 10.1050
def timetofloat(s):
    t = s.split(':')
    return float(t[0]+'.'+str(int(float(t[1])*100/60))+str(int(float(t[2])*100/60)))

# WADE - Get just the hour so that we can see how many vids happened over an hour period
def justthehour(s):
    t = s.split(':')
    return float(t[0])

# WADE - Get a count of how many vids are in that hour
#def hourvideocount(s):
#    print s
#    print hdDataframe['school'].shift(1)
#    if hdDataframe.school.eq(hdDataframe.school.shift()).all(): 
#    if s in hdDataframe['school'].shift():
#        x = 2
#    else:
#        x = 1
#    return (x)



# add float time column (easier for graphing)
hdDataframe['floatTime'] = hdDataframe['time'].apply(timetofloat)

#WADE - add hour to data
hdDataframe['hour'] = hdDataframe['time'].apply(justthehour)

#WADE - add hourvideocount
#hdDataframe['hour video count'] = hdDataframe['school'].apply(hourvideocount)
hdDataframe['hour video count'] = 1

#WADE concatenate the month and date and year
hdDataframe['full date'] = hdDataframe['month'].map(str) + ' ' + hdDataframe['date'].map(str) #+ ' ' + hdDataframe['year'].map(str)


#WADE - test hour video count
#hdDataframe['wade test'] = hdDataframe.school == hdDataframe.school.shift()

#Wade - test Johnny
key_columns = ['school', 'full date', 'hour']
#newframe_columns =['video folder', 'month', 'date', 'hour','hour video count' ]
#key_columns = ['date']
#hdDataframe['Sum of hourly video'] = hdDataframe.groupby(key_columns)['hour video count'].sum()
hvcDataframe = hdDataframe.groupby(key_columns,as_index=False)['hour video count'].count()
print (hvcDataframe) 

hvcDataframe.to_csv('hourly.csv')
#['hour video count'].sum())

# add file paths column
hdDataframe['full path'] = vidFullPaths

# check size
print(hdDataframe.shape)

# save hd1 or hd2
#if not hd2:
    # save hd1 dataframe as .pkl file
    #   hdDataframe.to_pickle('hd1Pickle')
    # save hd1 dataframe as .csv file
    #  hdDataframe.to_csv('hd1CSV')
#else:
    # save hd1 dataframe as .pkl file
#hdDataframe.to_pickle('hd2Pickle')
    # save hd1 dataframe as .csv file
hdDataframe.to_csv('baseline.csv')

# see how it looks (wont work if hd2 selected)
hdDataframe.head()


                           school full date  hour  hour video count
0             Agona Nkran Islamic    Feb 10   8.0                 6
1             Agona Nkran Islamic    Feb 10   9.0                 1
2             Agona Nkran Islamic    Feb 10  10.0                 8
3             Agona Nkran Islamic    Feb 10  11.0                 4
4             Agona Nkran Islamic    Feb 10  12.0                 8
5             Agona Nkran Islamic    Feb 10  13.0                 5
6             Agona Nkran Islamic    Feb 13   8.0                 1
7             Agona Nkran Islamic    Feb 13   9.0                 8
8             Agona Nkran Islamic    Feb 13  10.0                20
9             Agona Nkran Islamic    Feb 13  11.0                19
10            Agona Nkran Islamic    Feb 13  12.0                 9
11            Agona Nkran Islamic    Feb 13  13.0                 2
12            Agona Nkran Islamic    Feb 14   9.0                 6
13            Agona Nkran Islamic    Feb 14  10.

,hard drive,school,video folder,file name,day,month,date,time,year,floatTime,hour,hour video count,full date,full path
0,MasterTest,Agona Nkran Islamic,All,07_Feb_22_2017--Agona_Nkran_Islamic279063563.avi,Wed,Feb,22,07:45:20,2017,7.7533,7.0,1,Feb 22,/Volumes/GhanaStudyData/MasterTest/Agona Nkran...
1,MasterTest,Agona Nkran Islamic,All,07_Mar_01_2017--Agona_Nkran_Islamic494214773.avi,Wed,Mar,1,07:35:08,2017,7.5813,7.0,1,Mar 1,/Volumes/GhanaStudyData/MasterTest/Agona Nkran...
2,MasterTest,Agona Nkran Islamic,All,07_Mar_01_2017--Agona_Nkran_Islamic631361197.avi,Wed,Mar,1,07:26:24,2017,7.4340,7.0,1,Mar 1,/Volumes/GhanaStudyData/MasterTest/Agona Nkran...
3,MasterTest,Agona Nkran Islamic,All,08_Feb_09_2017--Agona_Nkran_Islamic280025593.avi,Thu,Feb,9,08:38:22,2017,8.6336,8.0,1,Feb 9,/Volumes/GhanaStudyData/MasterTest/Agona Nkran...
4,MasterTest,Agona Nkran Islamic,All,08_Feb_09_2017--Agona_Nkran_Islamic322518604.avi,Thu,Feb,9,08:27:10,2017,8.4516,8.0,1,Feb 9,/Volumes/GhanaStudyData/MasterTest/Agona Nkran...


## 3: Visualizing the Data

In [17]:

datapivottable = pd.pivot_table(hvcDataframe,index=['school', 'hour' ], columns = ['full date'], values = ['hour video count'], fill_value = 'XXXXXXXXXXXX', margins = True, margins_name = 'HOUR TOTAL', aggfunc=np.sum)

writer = pd.ExcelWriter('baselinevidecounts.xlsx')


 
for school in datapivottable.index.get_level_values(0).unique():
    temp_df = datapivottable.xs(school, level=0)
    temp_df.to_excel(writer,school)
    worksheet = writer.sheets[school]
    worksheet.set_column(1, 65, 12)  

    

print(writer)
writer.save()
